In [20]:
from sklearn.metrics import  make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
import pandas as pd
import numpy as np

# PREDICT THE AGE OF A BRAIN FROM MRI FEATURES

This task is primarily concerned with regression. However, we have perturbed the original MRI features in several ways. You will need to perform outlier detection, feature selection, and other preprocessing to achieve the best result.

The evaluation metric for this task is the Coefficient of Determination (R^2 ) Score which ranges from minus infinity to 1. 

`
from sklearn.metrics import r2_score
score = r2_score(y, y_pred)
`

Note: Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.


## Read In Data

In [146]:
X_test = pd.read_csv("./data/X_test.csv", index_col=0)
X_test.shape

(776, 832)

In [112]:
X_train = pd.read_csv("./data/X_train.csv", index_col=0)
y_train = pd.read_csv("./data/y_train.csv", index_col=0)
X_train.shape,y_train.shape

((1212, 832), (1212, 1))

In [113]:
X_train.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,NaN,1.144294e+06,785176.201298,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1.0,11.512994,832442.898114,NaN,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3.0,11.642076,NaN,NaN,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,785176.254867,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,NaN,102746.516920
4.0,10.407121,832442.831424,20585.557007,NaN,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,785176.190880,...,1.031009e+06,-1025.223865,18348.460040,NaN,NaN,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149


In [114]:
y_train.head()

,y
id,
0.0,71.0
1.0,73.0
2.0,66.0
3.0,55.0
4.0,67.0


In [148]:
X_test.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,9.101943,832442.839400,20585.538590,NaN,1.005104e+06,10.417525,597900.393691,10791.058148,9.829612e+05,785176.180458,...,1.049199e+06,-1002.286934,11388.219873,10.538011,9.070287,1122.095390,9.590537,1.006411e+06,NaN,106668.615874
1.0,11.722077,832442.826314,20585.509289,1039.097880,9.836399e+05,11.185000,597900.474419,10321.030961,9.998527e+05,785176.200371,...,1.063850e+06,-777.155703,15454.964631,10.438084,10.573977,830.040750,11.283446,1.064455e+06,100761.264268,NaN
2.0,11.200277,832442.820359,20585.511136,1081.926822,1.059913e+06,NaN,597900.432849,9394.048739,8.439144e+05,785176.185410,...,1.033983e+06,-866.209778,13616.308487,10.123607,8.273391,926.072523,9.277080,1.007427e+06,106440.456728,103405.273232
3.0,9.668873,832442.820901,20585.528528,1063.771791,1.023284e+06,9.635705,597900.426803,10640.030819,1.018818e+06,785176.263405,...,1.064161e+06,NaN,14596.359851,10.756986,8.540542,1067.003746,9.803903,1.076560e+06,NaN,100531.960204
4.0,10.329962,832442.871842,20585.483009,1013.321073,9.396156e+05,10.664417,597900.399834,10464.062039,1.024615e+06,785176.251995,...,1.070782e+06,-1033.312644,14321.185784,10.708867,10.285677,985.074197,9.284969,1.097973e+06,109797.625066,104849.648797


In [147]:
print('In the test set there are missing: %d' % sum(np.isnan(X_test.values).flatten()))

In the test set there are missing: 40610


In [115]:
X_train.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
count,1118.000000,1114.000000,1117.000000,1106.000000,1.117000e+03,1128.000000,1105.000000,1127.000000,1.116000e+03,1124.000000,...,1.134000e+03,1125.000000,1098.000000,1121.000000,1120.000000,1109.000000,1115.000000,1.112000e+03,1124.000000,1091.000000
mean,10.026057,832442.859290,20585.524887,1048.958235,1.000291e+06,10.085010,597900.429955,10389.657239,9.998422e+05,785176.225858,...,1.049674e+06,-876.044006,13492.600186,10.554762,10.057767,1066.141107,10.008269,1.050199e+06,99798.480171,104903.905758
std,0.968347,0.028258,0.029051,28.430733,9.740891e+04,0.968026,0.028128,1655.843472,1.022441e+05,0.028799,...,2.839579e+04,164.585576,2519.835006,0.283844,0.982656,226.606986,1.018930,2.814210e+04,9576.128720,2768.405350
min,6.672068,832442.808579,20585.473809,1000.063783,6.800215e+05,6.984052,597900.381003,3644.074892,6.095730e+05,785176.176297,...,1.000105e+06,-1597.766964,2536.030655,10.010366,6.841039,496.007706,6.466963,1.000002e+06,73207.994891,100012.896777
25%,9.381273,832442.835941,20585.501013,1024.969967,9.360882e+05,9.470582,597900.406110,9339.537887,9.322937e+05,785176.201279,...,1.025054e+06,-975.398714,11947.954006,10.321039,9.379001,899.067501,9.325229,1.027575e+06,93416.252400,102596.190683
50%,10.000079,832442.860041,20585.524817,1047.985497,1.000557e+06,10.089601,597900.429787,10295.013382,1.001261e+06,785176.225608,...,1.049296e+06,-875.508235,13352.186179,10.554260,10.114370,1049.027077,10.005684,1.050262e+06,99802.127899,104846.235709
75%,10.664998,832442.882951,20585.550525,1073.180317,1.064617e+06,10.752707,597900.452983,11304.073469,1.068359e+06,785176.250421,...,1.074354e+06,-773.174562,14893.726023,10.792195,10.745370,1215.057985,10.658120,1.073831e+06,106400.748441,107098.669350
max,12.956099,832442.908334,20585.573514,1099.977638,1.331630e+06,12.747734,597900.480810,17347.531573,1.284804e+06,785176.276168,...,1.099771e+06,-281.030205,24815.260375,11.091050,13.530204,2122.032859,13.163113,1.099918e+06,130694.436443,109984.169649


In [145]:
print('Missing: %d' % sum(np.isnan(X_train.values).flatten()))

Missing: 76910


So there is missing values in the training set. Question: are there missing values in the test set?

In [117]:
print('Missing: %d' % sum(np.isnan(y_train.values).flatten()))

Missing: 0


No missing values in the training labels

# Preprocess
Todo:
- deal with missing data (NAN) (https://machinelearningmastery.com/handle-missing-data-python/ )  
- outlier detection, remove outliers (https://machinelearningmastery.com/model-based-outlier-detection-and-removal-in-python/)
- normalize (standardnormalization)

### 1.) Deal with missing data

#### a.) Remove Rows With Missing Values:

In [118]:
X_train_subset = X_train.dropna(inplace=False)
X_train_subset.shape

(0, 832)

So we cannot use this strategy, because all rows are removed

#### b.) Impute Missing values

In [119]:
from sklearn.impute import SimpleImputer

##### b.1) replace missing values with the mean value for each column
Could also try median or mode

In [120]:
X_train_mean_imputed = X_train.fillna(X_train.mean(), inplace=False)
X_train_mean_imputed.shape

(1212, 832)

In [121]:
X_train_mean_imputed.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
id,,,,,,,,,,,,,,,,,,,,,
0.0,10.891876,832442.812375,20585.544083,1028.369495,1.163780e+06,9.199135,597900.477629,10389.657239,1.144294e+06,785176.201298,...,1.024198e+06,-855.549602,12176.073427,10.647729,10.916371,1220.065443,8.566724,1.036263e+06,85338.558539,103088.664210
1.0,11.512994,832442.898114,20585.524887,1012.624877,1.028911e+06,10.906408,597900.458612,8127.016078,1.099166e+06,785176.258299,...,1.086806e+06,-787.397942,10493.095660,10.586492,9.463962,917.094909,10.231822,1.007163e+06,95695.020645,105161.109422
2.0,11.052185,832442.896307,20585.512844,1003.953827,9.231756e+05,9.212979,597900.426764,10738.092422,1.027863e+06,785176.223468,...,1.018533e+06,-906.997242,10959.516944,10.769287,10.342160,637.027802,10.705461,1.019955e+06,80253.299882,104177.051666
3.0,11.642076,832442.859290,20585.524887,1004.672084,9.459461e+05,9.553420,597900.450367,13524.096973,1.168144e+06,785176.254867,...,1.047017e+06,-1011.742516,16845.309819,10.483830,10.594941,1114.069590,10.321063,1.085442e+06,99798.480171,102746.516920
4.0,10.407121,832442.831424,20585.557007,1048.958235,9.957182e+05,8.419164,597900.423639,12894.065081,1.063199e+06,785176.190880,...,1.031009e+06,-1025.223865,18348.460040,10.554762,10.057767,1230.088215,10.250096,1.024812e+06,101815.745499,105163.749149


In [122]:
X_train_mean_imputed.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x822,x823,x824,x825,x826,x827,x828,x829,x830,x831
count,1212.000000,1212.000000,1212.000000,1212.000000,1.212000e+03,1212.000000,1212.000000,1212.000000,1.212000e+03,1212.000000,...,1.212000e+03,1212.000000,1212.000000,1212.000000,1212.000000,1212.000000,1212.000000,1.212000e+03,1212.000000,1212.000000
mean,10.026057,832442.859290,20585.524887,1048.958235,1.000291e+06,10.085010,597900.429955,10389.657239,9.998422e+05,785176.225858,...,1.049674e+06,-876.044006,13492.600186,10.554762,10.057767,1066.141107,10.008269,1.050199e+06,99798.480171,104903.905758
std,0.930005,0.027090,0.027888,27.157959,9.351014e+04,0.933849,0.026856,1596.674554,9.810785e+04,0.027733,...,2.746609e+04,158.563365,2398.299115,0.272971,0.944592,216.755988,0.977271,2.695513e+04,9221.631883,2626.460400
min,6.672068,832442.808579,20585.473809,1000.063783,6.800215e+05,6.984052,597900.381003,3644.074892,6.095730e+05,785176.176297,...,1.000105e+06,-1597.766964,2536.030655,10.010366,6.841039,496.007706,6.466963,1.000002e+06,73207.994891,100012.896777
25%,9.440852,832442.838507,20585.503515,1027.518864,9.427006e+05,9.518125,597900.408556,9409.300050,9.430492e+05,785176.202866,...,1.027129e+06,-965.633580,12164.233176,10.338268,9.460857,914.763005,9.384658,1.029501e+06,94149.196760,102892.311899
50%,10.026057,832442.859290,20585.524887,1048.958235,1.000291e+06,10.085010,597900.429955,10389.657239,9.998422e+05,785176.225858,...,1.049674e+06,-876.044006,13492.600186,10.554762,10.057767,1066.141107,10.008269,1.050199e+06,99798.480171,104903.905758
75%,10.624582,832442.880460,20585.547770,1070.599035,1.057470e+06,10.689486,597900.451121,11212.573544,1.060764e+06,785176.249144,...,1.072653e+06,-785.801099,14700.041808,10.768303,10.663939,1194.283142,10.603448,1.071379e+06,105761.210966,106826.950763
max,12.956099,832442.908334,20585.573514,1099.977638,1.331630e+06,12.747734,597900.480810,17347.531573,1.284804e+06,785176.276168,...,1.099771e+06,-281.030205,24815.260375,11.091050,13.530204,2122.032859,13.163113,1.099918e+06,130694.436443,109984.169649


In [123]:
np.sum(np.sum(X_train_mean_imputed.isna()))

0

We see there are no NAN values anymore

The same but with SimpleImputer:

In [124]:
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
X_train_val = X_train.values
mean_imputed_values = imputer.fit_transform(X_train_val)
print('Missing: %d' % np.isnan(mean_imputed_values).sum())

Missing: 0


##### b.2) Impute with zero

In [125]:
imputer = SimpleImputer(missing_values=np.nan , strategy="constant", fill_value=0)
X_train_val = X_train.values
zero_imputed_values = imputer.fit_transform(X_train_val)
print('Missing: %d' % np.isnan(zero_imputed_values).sum())

Missing: 0


### 2.) Outlier detection

" Each method will be defined, then fit on the training dataset. The fit model will then predict which examples in the training dataset are outliers and which are not (so-called inliers). The outliers will then be removed from the training dataset, then the model will be fit on the remaining examples and evaluated on the entire test dataset.

It would be invalid to fit the outlier detection method on the entire training dataset as this would result in data leakage. That is, the model would have access to data (or information about the data) in the test set not used to train the model. This may result in an optimistic estimate of model performance."

Therefore this needs to be done in the pipeline, I think before standardscaler or other normalization methods



#### 2.1) Isolation Forest

In [94]:
from sklearn.ensemble import IsolationForest

#### 2.2) Local Outlier Factor

A simple approach to identifying outliers is to locate those examples that are far from the other examples in the feature space.

In [126]:
# identify outliers in the training dataset
lof = LocalOutlierFactor()
outliers = lof.fit_predict(mean_imputed_values)

In [133]:
 # select all rows that are not outliers
mask = outliers != -1
X_train_outlierfree, y_train_outlierfree = X_train.values[mask, :], y_train.values[mask]
# summarize the shape of the updated training dataset
print(X_train_outlierfree.shape, y_train_outlierfree.shape)

(1159, 832) (1159, 1)


In [135]:
print("there are {} outliers ".format(np.sum(outliers == -1)))

there are 53 outliers 


### 3.) Normalize


In [83]:
from sklearn.preprocessing import StandardScaler



## Models

In [75]:
from sklearn.metrics import make_scorer, r2_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

#  The pipeline can be used as any other estimator
# and avoids leaking the test set into the train set

my_score = make_scorer(r2_score)

### 1. ) Baseline Ridge on mean imputed data

In [87]:
# I think the problem here is that we impute the whole trainig set first, and then do cross validation 
# --> leaking the train set into the test set?


# ridge = Ridge()
# pipeline = Pipeline(steps=[('imputer', imputer),('model', ridge)])
# kfold = KFold(n_splits=10, shuffle=True, random_state=1)
# result = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring=my_score)
# print('mean R2 score: %.3f' % result.mean())

In [88]:
# I think the problem here is that we impute the whole trainig set first, and then do cross validation 
# --> leaking the train set into the test set?


# ridge_m1 = Ridge()
# param = {"alpha" : [1e-15, 1e-4, 1, 20]}
# ridge_regression = GridSearchCV(ridge_m1, param, scoring= my_score, cv= 5)
# ridge_regression.fit(mean_imputed_values, y_train)

# print(ridge_regression.best_params_)
# print(ridge_regression.best_score_)

In [92]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
ridge = Ridge()
pipeline = Pipeline(steps=[('imputer', imputer),('ridge', ridge)])
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20, 50, 100]}


search = GridSearchCV(pipeline, param_grid, n_jobs=-1, cv= 10)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=-0.497):
{'ridge__alpha': 100}


/Users/sarahmorillo/anaconda3/envs/python_hs2020/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.57833e-46): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,


Observation : Ill-conditioned matrix. What does this mean? Not full rank? -> feauter selection should be done prior?

### 2.) Ridge, cross validation, mean imputed data, standard normalization
standard normalization : z = (x - u) / s ( u: mean, s:std)

In [93]:
# the same as above but with standard scaler
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
ridge = Ridge()
pipeline = Pipeline(steps=[('imputer', imputer),('scaler', StandardScaler()), ('ridge', ridge)])
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20, 50, 100, 200]}
param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20]}

search = GridSearchCV(pipeline, param_grid, n_jobs=-1, cv= 10)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Best parameter (CV score=0.152):
{'ridge__alpha': 200}


Observation: ridge always want's the alpha to be as large as possible, i.e. it always chooses the largest possible alpha -> does this mean, the model is not fitting well? 

### 3.) Ridge, cross validation, mean imputed, standard normalized, outlier detection

In [101]:
# https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline

## Not working
from sklearn.pipeline import Pipeline, TransformerMixin
from sklearn.neighbors import LocalOutlierFactor

class OutlierExtractor(TransformerMixin):
    def __init__(self, **kwargs):
        """
        Create a transformer to remove outliers. A threshold is set for selection
        criteria, and further arguments are passed to the LocalOutlierFactor class

        Keyword Args:
            neg_conf_val (float): The threshold for excluding samples with a lower
               negative outlier factor.

        Returns:
            object: to be used as a transformer method as part of Pipeline()
        """

        self.threshold = kwargs.pop('neg_conf_val', -10.0)

        self.kwargs = kwargs

    def transform(self, X, y):
        """
        Uses LocalOutlierFactor class to subselect data based on some threshold

        Returns:
            ndarray: subsampled data

        Notes:
            X should be of shape (n_samples, n_features)
        """
        X = np.asarray(X)
        y = np.asarray(y)
        lcf = LocalOutlierFactor(**self.kwargs)
        lcf.fit(X)
        return (X[lcf.negative_outlier_factor_ > self.threshold, :],
                y[lcf.negative_outlier_factor_ > self.threshold])

    def fit(self, *args, **kwargs):
        return self
    
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
ridge = Ridge()
pipeline = Pipeline(steps=[('imputer', imputer),('outliers', OutlierExtractor()),('scaler', StandardScaler()), ('ridge', ridge)])

param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20, 50, 100, 200]}
param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20]}

search = GridSearchCV(pipeline, param_grid, n_jobs=-1, cv= 10)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)



TypeError: transform() missing 1 required positional argument: 'y'

In [137]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
ridge = Ridge()
pipeline = Pipeline(steps=[('imputer', imputer),('scaler', StandardScaler()), ('ridge', ridge)])

param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20, 50, 100, 200]}
# param_grid = {'ridge__alpha': [1e-15, 1e-4, 1, 20]}

search = GridSearchCV(pipeline, param_grid, n_jobs=-1, cv= 10)
search.fit(X_train_outlierfree, y_train_outlierfree)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.233):
{'ridge__alpha': 200}


## Best found model

For now the best found model is the last one: 3.) Ridge, cross validation, mean imputed, standard normalized, outlier detection.


Fit model to whole training set and predict on test set

In [175]:
# 1.) impute missing values
imputer = SimpleImputer(missing_values=np.nan , strategy='mean')
X_train_val = X_train.values
X_train_imputed = imputer.fit_transform(X_train_val)


## Question: impute X_test with same immputer, or should we use other imputer fited to test data?
## For now: use same
X_test_imputed = imputer.transform(X_test.values)
## Question: should X_test also be normalized? Or should normalization be done on Xtest and X_train together?
## https://datascience.stackexchange.com/questions/53138/which-comes-first-multiple-imputation-splitting-into-train-test-or-standardiz
## Performing pre-processing before splitting will mean that information from your 
## test set will be present during training, causing a data leak.
##
##The key here is that you are learning everything from the training set and then "predicting" on to the test set. 

# 2.) remove outliers
# identify outliers in the training dataset
lof = LocalOutlierFactor()
outliers = lof.fit_predict(X_train_imputed)
# select all rows that are not outliers
mask = outliers != -1
X, y = X_train_imputed[mask, :], y_train[mask]

# Don't think I have to remove outliers of dataset, otherwise will have less rows in output...

# 3.) normalize
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)
X_test_norm = scaler.transform(X_test_imputed) # transforming with values learned on X_train

ridge = Ridge(alpha=200)
ridge.fit(X_norm, y)

# prediction in train set
y_train_pred = ridge.predict(X_norm)
print("Score on training set: {}".format(r2_score(y_true=y, y_pred= y_train_pred)))

# predict on test set
y_pred = ridge.predict(X_test_norm)


# output
output_csv = pd.concat([pd.Series(X_test.index.values), pd.Series(y_pred.flatten())], axis=1)
output_csv.columns = ["id", "y"]

pd.DataFrame.to_csv(output_csv, "./data/submit.csv", index=False)


Score on training set: 0.769847815885915
